In [ ]:
import habitat_sim
from habitat_sim.utils import common as utils
from habitat_sim.utils import viz_utils as vut
from habitat_sim.utils.common import d3_40_colors_rgb

import gzip
import json
import random
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import font_manager
# font_manager.fontManager.addfont("~/miniconda3/envs/vlnce/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/Arial-Bold.ttf")
# font_manager.fontManager.addfont("~/miniconda3/envs/vlnce/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/Arial.ttf")
plt.rcParams['font.sans-serif'] = ["Arial"]
plt.rcParams['axes.unicode_minus'] = False

import seaborn as sns
from PIL import Image

import numpy as np
import pickle
import habitat

In [ ]:
def load_score(pkl_path="/home/ubuntu/MLA/score_save_single.pkl"):
    with open(pkl_path, "rb") as f:
        score_final = pickle.load(f)
    for k,v in score_final.items():
        s = score_final[k]["score"]
        score_final[k]["score"] = s/np.sum(s, axis=1, keepdims=True)
    score_success = {k:v for k, v in score_final.items() if v["infos"]["success"]==1 and len(v["score"])>0}
    score_fail = {k:v for k, v in score_final.items() if v["infos"]["success"]==0 and len(v["score"])>0}
    return score_success, score_fail
def load_action(pkl_path="/home/ubuntu/MLA/action_save_single.pkl"):
    with open(pkl_path, "rb") as f:
        action_final = pickle.load(f)
    return action_final

In [ ]:
s1, s2 = load_score()
action_final = load_action()
split = "train"
sub_data_path = "/home/ubuntu/MLA/data/datasets/R2R_VLNCE_NRSub/%s/%s_sub.json.gz"%(split,split)
with gzip.open(sub_data_path, "r") as f:
    sub_data = json.loads(f.read())
sub_gt_path = "/home/ubuntu/MLA/data/datasets/R2R_VLNCE_NRSub/%s/%s_gt.json.gz"%(split,split)
with gzip.open(sub_gt_path, "r") as f:
    sub_gt = json.loads(f.read())

Success

In [ ]:
# 10831,6519 nice
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import gzip
import json

ylabels = ['',
 '',
 '']
xticks = np.arange(0,52,5)+0.5
xlabels = [str(v) for v in range(0,52,5)]
ep_id = 10831
sample = s1[ep_id]
fig = plt.figure(figsize=(16,2),dpi=300)
ax = fig.add_subplot(111)
sns.heatmap(sample["score"].transpose(1,0),annot=False,fmt='.2f',cmap="YlOrRd",linecolor="",linewidths=0,cbar=False)
ax.set_xlabel("steps", fontsize=13, fontweight='bold')
plt.xticks(fontsize=12, ticks=xticks,labels=xlabels) 
plt.yticks(fontsize=12, labels=ylabels,ticks=[0.5,1.5,2.5],rotation='horizontal',stretch=1)
plt.savefig("success.svg", bbox_inches='tight', dpi=300)
plt.show()
print(sample["infos"])

Failure

In [ ]:
# ylabels = ['',
#  '',
#  '']
epfail_id = 5545
sample = s1[epfail_id]
xticks = np.arange(0,sample["score"].shape[0],5)+0.5
xlabels = [str(v) for v in range(0,sample["score"].shape[0],5)]
fig = plt.figure(figsize=(16,2),dpi=300)
ax = fig.add_subplot(111)
sns.heatmap(sample["score"].transpose(1,0),annot=False,fmt='.2f',cmap="YlOrRd",linecolor="",linewidths=0,cbar=False)
ax.set_xlabel("steps", fontsize=13, fontweight='bold')
plt.xticks(fontsize=12, ticks=xticks,labels=xlabels) 
plt.yticks(fontsize=12, )
plt.savefig("fail1.svg", bbox_inches='tight', dpi=300)
plt.show()
print(sample["infos"])

In [ ]:
epfail_id = 10778
# s1, s2 = load_score(pkl_path="/home/ubuntu/MLA/score_t10778.pkl")
# action_final = load_action(pkl_path="/home/ubuntu/MLA/action_t10778.pkl")
sample = s2[epfail_id]
xticks = np.arange(0,sample["score"].shape[0],5)+0.5
xlabels = [str(v) for v in range(0,sample["score"].shape[0],5)]
fig = plt.figure(figsize=(16,2),dpi=300)
ax = fig.add_subplot(111)
sns.heatmap(sample["score"].transpose(1,0),annot=False,fmt='.2f',cmap="YlOrRd",linecolor="",linewidths=0,cbar=False)
ax.set_xlabel("steps", fontsize=13, fontweight='bold')
plt.xticks(fontsize=12, ticks=xticks,labels=xlabels) 
plt.yticks(fontsize=12, )
plt.savefig("fail2.svg", bbox_inches='tight', dpi=300)
plt.show()
print(sample["infos"])

In [ ]:
ep_id = 5545
sample_ep = None
for v in sub_data["episodes"]:
    if v["episode_id"]==ep_id:
        sample_ep = v
sample_ep["actions"] = action_final[ep_id]

Simulator

In [ ]:
test_scene = "/home/ubuntu/MLA/data/scene_datasets/"+sample_ep["scene_id"]
start_point = np.array(sample_ep["start_position"])
start_rotation = np.array(sample_ep["start_rotation"])
end_point = np.array(sample_ep["goals"][0]["position"])
sim_settings = {
    "width": 512,  # Spatial resolution of the observations    
    "height": 512,
    "scene": test_scene,  # Scene path
    "default_agent": 0,  
    "sensor_height": 1.5,  # Height of sensors in meters
    "color_sensor": True,  # RGB sensor
    "semantic_sensor": True,  # Semantic sensor
    "depth_sensor": True,  # Depth sensor
    "seed": 1,
}
trajectory_id = ep_id
def display_sample(rgb_obs, semantic_obs, depth_obs, idx=0):
    rgb_img = Image.fromarray(rgb_obs, mode="RGBA")
    rgb_img.save("tmp/trajectory%d_%d_rgb.png"%(trajectory_id,idx))
    semantic_img = Image.new("P", (semantic_obs.shape[1], semantic_obs.shape[0]))
    semantic_img.putpalette(d3_40_colors_rgb.flatten())
    semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
    semantic_img = semantic_img.convert("RGBA")
    # semantic_img.save("tmp/trajectory%d_%d_semantic.png"%(trajectory_id,idx))
    
    depth_img = Image.fromarray((depth_obs / 10 * 255).astype(np.uint8), mode="L")
    # depth_img.save("tmp/trajectory%d_%d_depth.png"%(trajectory_id,idx))

    arr = [rgb_img, semantic_img, depth_img]
    titles = ['rgb', 'semantic', 'depth']
    # plt.figure(figsize=(12 ,8))
    # for i, data in enumerate(arr):
    #     ax = plt.subplot(1, 3, i+1)
    #     ax.axis('off')
    #     ax.set_title(titles[i])
    #     plt.imshow(data)
    # plt.show()

In [ ]:
trajectory_id

In [ ]:
def make_cfg(settings):
    sim_cfg = habitat_sim.SimulatorConfiguration()
    sim_cfg.gpu_device_id = 0
    print(sim_cfg)
    sim_cfg.scene_id = settings["scene"]
    
    # Note: all sensors must have the same resolution
    sensors = {
        "color_sensor": {
            "sensor_type": habitat_sim.SensorType.COLOR,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
        },
        "depth_sensor": {
            "sensor_type": habitat_sim.SensorType.DEPTH,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
        },
        "semantic_sensor": {
            "sensor_type": habitat_sim.SensorType.SEMANTIC,
            "resolution": [settings["height"], settings["width"]],
            "position": [0.0, settings["sensor_height"], 0.0],
        },  
    }
    
    sensor_specs = []
    for sensor_uuid, sensor_params in sensors.items():
        if settings[sensor_uuid]:
            sensor_spec = habitat_sim.SensorSpec()
            sensor_spec.uuid = sensor_uuid
            sensor_spec.sensor_type = sensor_params["sensor_type"]
            sensor_spec.resolution = sensor_params["resolution"]
            sensor_spec.position = sensor_params["position"]

            sensor_specs.append(sensor_spec)
            
    # Here you can specify the amount of displacement in a forward action and the turn angle
    agent_cfg = habitat_sim.agent.AgentConfiguration()
    agent_cfg.sensor_specifications = sensor_specs
    agent_cfg.action_space = {
        "stop": habitat_sim.agent.ActionSpec(
            "move_forward", habitat_sim.agent.ActuationSpec(amount=0)
        ),
        "move_forward": habitat_sim.agent.ActionSpec(
            "move_forward", habitat_sim.agent.ActuationSpec(amount=0.25)
        ),
        "turn_left": habitat_sim.agent.ActionSpec(
            "turn_left", habitat_sim.agent.ActuationSpec(amount=15.0)
        ),
        "turn_right": habitat_sim.agent.ActionSpec(
            "turn_right", habitat_sim.agent.ActuationSpec(amount=15.0)
        ),
    }
    
    return habitat_sim.Configuration(sim_cfg, [agent_cfg])
# del sim
cfg = make_cfg(sim_settings)
sim = habitat_sim.Simulator(cfg)

In [ ]:
random.seed(sim_settings["seed"])
sim.seed(sim_settings["seed"])

# Set agent state
agent = sim.initialize_agent(sim_settings["default_agent"])
agent_state = habitat_sim.AgentState()
agent_state.position = start_point
agent_state.rotation = start_rotation
agent.set_state(agent_state)
observations = sim.get_sensor_observations()
rgb = observations["color_sensor"]
semantic = observations["semantic_sensor"]
depth = observations["depth_sensor"]
    
display_sample(rgb, semantic, depth, idx=0)
# Get agent state
agent_state = agent.get_state()
print("agent_state: position", agent_state.position, "rotation", agent_state.rotation)

action_mapping = ["stop", "move_forward", "turn_left", "turn_right"]
total_frames = 0
action_names = list(
    cfg.agents[
        sim_settings["default_agent"]
    ].action_space.keys()
)
max_frames = 5

for i, action in enumerate(action_final[ep_id]):
    action = action_mapping[action]
    print("action", action)
    observations = sim.step(action)
    rgb = observations["color_sensor"]
    semantic = observations["semantic_sensor"]
    depth = observations["depth_sensor"]
    
    display_sample(rgb, semantic, depth, idx=i+1)
    
    total_frames += 1

In [ ]:
sample_ep["sub_instruction"]

In [34]:
ep_id = 10831
sample_ep = None
for v in sub_data["episodes"]:
    if v["episode_id"]==ep_id:
        sample_ep = v